<a href="https://colab.research.google.com/github/LikaOkudzhava/BrainyScan/blob/main/KerasTuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import zipfile
from google.colab import drive
gdrive = '/content/drive'
drive.mount(gdrive, force_remount=False)
gdisk = os.path.join(gdrive, 'MyDrive')
filename = 'smallpreprocessed'
archive_path = os.path.join(gdisk, filename)
with zipfile.ZipFile('/content/drive/MyDrive/smallpreprocessed.zip', 'r') as zip_ref:
 zip_ref.extractall(f'{filename}_extracted')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
!pip install keras_tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.4 MB/s eta 0:00:00


In [15]:


import cv2

import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from tqdm import tqdm

import keras
from keras.callbacks import EarlyStopping,ModelCheckpoint

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint,TensorBoard,LambdaCallback
from keras.layers import Input,Dropout, Dense,GlobalAveragePooling2D, BatchNormalization
from keras.models import Sequential,Model
from keras.applications.resnet import ResNet50
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf
from tqdm import tqdm
import seaborn as sns
import numpy as np
import itertools
import datetime
import keras_tuner as kt


In [12]:
dataset_dir = '/content/smallpreprocessed_extracted/data/SmallPreprocessed'
image_size = (224, 224)
batch_size = 32

datagen = ImageDataGenerator(
    preprocessing_function= tf.keras.applications.resnet.preprocess_input,
    rescale=1./255,
    horizontal_flip=True
)

train_generator = datagen.flow_from_directory(
    os.path.join(dataset_dir, 'train'),
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle = True
)

test_generator = datagen.flow_from_directory(
    os.path.join(dataset_dir, 'test'),
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle = False
)

val_generator = datagen.flow_from_directory(
    os.path.join(dataset_dir, 'val'),
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'categorical',
    shuffle = True)

Found 12000 images belonging to 4 classes.
Found 2000 images belonging to 4 classes.
Found 2000 images belonging to 4 classes.


In [19]:
def build_model(hp):
    model = Sequential()

    # Add the pre-trained ResNet50 base
    # freezing it initially, as it's the standard for transfer learning base
    # pooling='avg' handles the global average pooling
    resnet_base = ResNet50(
        input_shape=(image_size[0], image_size[1], 3),
        include_top=False,
        pooling='avg',
        weights='imagenet'
    )
    resnet_base.trainable = False # Freeze the base model
    model.add(resnet_base)

    # first dense layer
    hp_dense1 = hp.Choice(
        'dense1', # Name of the hyperparameter
        values=[512, 1024, 2048] # The values to try for the first dense layer
    )
    model.add(Dense(units=hp_dense1, activation='relu'))
    model.add(BatchNormalization()) # Keep BatchNormalization ?

   # second dense layer
    hp_dense2 = hp.Choice(
        'dense2',
        values=[256, 512, 1024] # The values to try for the second dense layer
    )
    model.add(Dense(units=hp_dense2, activation='relu'))
    model.add(BatchNormalization())

    # dropout rates
    hp_dropout_rate = hp.Choice(
        'dropout_rate',
        values=[0.2, 0.3, 0.4, 0.5]
    )
    model.add(Dropout(hp_dropout_rate))


    # Final output layer
    model.add(Dense(num_classes, activation='softmax'))

   # learning rate
    hp_learning_rate = hp.Choice(
        'learning_rate',
        values=[0.0001, 0.00005, 0.00001]
    )

    optimizer = keras.optimizers.Adam(learning_rate=hp_learning_rate)

    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model



In [ ]:
# Instantiate the KerasTuner


num_classes = train_generator.num_classes

tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=20, # try 20 different combinations of hyperparameters
    executions_per_trial=1, # each trial trains the model once
    directory='keras_tuner_dir', # Folder to save tuning results
    project_name='mri_resnet_tuning'
)

# Print a summary of the search space
tuner.search_space_summary()


print("\nStarting hyperparameter search...\n")
tuner.search(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=[
        keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
        keras.callbacks.ModelCheckpoint("tuned_model_temp.keras", save_best_only=True, monitor='val_accuracy', mode='max')

    ]
)

# --- 4. Get the best model and print results ---
print("\nHyperparameter search complete.\n")
tuner.results_summary()

# Get the best model(s)
best_models = tuner.get_best_models(num_models=1)
best_model = best_models[0]

print("\nBest model summary:")
best_model.summary()

print("\nEvaluating the best model on test data:")
loss, accuracy = best_model.evaluate(test_generator)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

# You can save the final best model
best_model.save("best_mri_resnet_model.keras")

Search space summary
Default search space size: 4
dense1 (Choice)
{'default': 512, 'conditions': [], 'values': [512, 1024, 2048], 'ordered': True}
dense2 (Choice)
{'default': 256, 'conditions': [], 'values': [256, 512, 1024], 'ordered': True}
dropout_rate (Choice)
{'default': 0.2, 'conditions': [], 'values': [0.2, 0.3, 0.4, 0.5], 'ordered': True}
learning_rate (Choice)
{'default': 0.0001, 'conditions': [], 'values': [0.0001, 5e-05, 1e-05], 'ordered': True}

Starting hyperparameter search...


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
2048              |2048              |dense1
1024              |1024              |dense2
0.5               |0.5               |dropout_rate
5e-05             |5e-05             |learning_rate

Epoch 1/20
242/375 ━━━━━━━━━━━━━━━━━━━━ 13:17 6s/step - accuracy: 0.4010 - loss: 1.6858